In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import display_html, HTML, display
css = !cat styles.css
display(HTML(''.join(css)))

import numpy as np

import display_helpers

In [2]:
from erafixer import EraFixer, COL_LOOKUP

In [3]:
# Excel file
# fn = 'ERA2018_Pubs_02_27-Oct-17.xlsx'
fn = 'all.xlsx'

### Command Line Help

```
➜  python erafixer.py --help
usage: erafixer.py [-h] [--detect_author AUTHOR] [--detect_journal JOURNAL]
                   [--set_discipline DISCIPLINE] [--split_disciplines]
                   [--prefix PREFIX] [--carry_forward_forcs]
                   [--set_forc FORC_STRING] [--justify JUSTIFY_STRING]
                   [--sheet_index SHEET_INDEX] [--verbose]
                   ERAFILE

Process and update ERA codes

positional arguments:
  ERAFILE               ERA file as excel spreadsheet

optional arguments:
  -h, --help            show this help message and exit
  --detect_author AUTHOR
                        Part of the author name in AUTHOR column, should be
                        unique substring
  --detect_journal JOURNAL
                        Part of the author name in JOURNAL column, should be
                        unique substring
  --set_discipline DISCIPLINE
                        Discipline to be set
  --split_disciplines   Split ERAFILE into different files called
                        <PREFIX>_<DISC>.xlsx for each discipline
  --prefix PREFIX       Prefix for split-disciplines
  --carry_forward_forcs
                        Carry 2015 codes forward into the corresponding 2018
                        columns
  --set_forc FORC_STRING
                        Apply the FORC string
  --justify JUSTIFY_STRING
                        Justification string [optional for --set_forc]
  --sheet_index SHEET_INDEX
                        Excel sheet to use, defaults to first sheet
  --verbose             Show some output, default false

```

### Create EraFixer instance

Note the `sheet_index=1` passed to constructor. If not passed and more than one sheet is present, prompt will ask for index number (works in notebook and cli)

In [4]:
erafixer = EraFixer(fn=fn, sheet_index=1, verbose=True)

Parsing file all.xlsx
Using sheet index 1 - Tab 1 for 02


### Set Discipline

Corresponds to the `--detect_author` and `--detect_journal` options

#### By Journal

In [5]:
journal_list = [
    ('astrophysical', 'astro'),
    ('geophysics', 'geo'),
    ('optics express', 'photonics')
]

In [6]:
for journal in journal_list:
    erafixer.set_journal_discipline(journal[0], journal[1])

Setting discipline to 'astro' for 'astrophysical' on 159 rows
Setting discipline to 'geo' for 'geophysics' on 4 rows
'geo' not in PhysAstro, setting HANDLED=1
Setting discipline to 'photonics' for 'optics express' on 96 rows


#### By Author

In [7]:
# WARNING: This generates a false positive, see row 1 of disc_astro.xlsx after split
erafixer.set_author_discipline('marco', 'astro')

Setting discipline to 'astro' for 'marco' on 52 rows


In [8]:
author_list = [
    ('zvyagin', 'biophotonics'),
    ('schwab', 'astro'),
    ('gee', 'astro'),
    ('mcgee', 'astro'),
    ('steel', 'quantum'),
    ('steele', 'astro'),
    ('casteels', 'astro'),
    ('spence', 'photonics')
]

In [9]:
for author in author_list:
    erafixer.set_author_discipline(author[0], author[1])

Setting discipline to 'biophotonics' for 'zvyagin' on 37 rows
Setting discipline to 'astro' for 'schwab' on 30 rows
Setting discipline to 'astro' for 'gee' on 1 rows
Setting discipline to 'astro' for 'mcgee' on 1 rows
Setting discipline to 'quantum' for 'steel' on 86 rows
Setting discipline to 'astro' for 'steele' on 6 rows
Setting discipline to 'astro' for 'casteels' on 1 rows
Setting discipline to 'photonics' for 'spence' on 30 rows


#### Save
There is no explicit save for the commands above. The cli interface **will** automatically save back to spreadsheet when given the `--detect_author` or `--detect_journal` options

In [10]:
erafixer.save()

File saved: all.xlsx


'all.xlsx'

#### Split Disciplines

Note: There could also feasibly be a `--detect_discipline` command that would work against the giant spreadsheet. 

In [11]:
erafixer.split_disciplines('disc')

File saved: disc_quantum.xlsx
File saved: disc_geo.xlsx
File saved: disc_astro.xlsx
File saved: disc_biophotonics.xlsx
File saved: disc_photonics.xlsx


['disc_quantum.xlsx',
 'disc_geo.xlsx',
 'disc_astro.xlsx',
 'disc_biophotonics.xlsx',
 'disc_photonics.xlsx']

#### Carry Forward FORCS

In [12]:
astro_erafixer = EraFixer(fn='disc_astro.xlsx', verbose=True)

Parsing file disc_astro.xlsx
Using sheet index 0 - Sheet1


In [13]:
astro_erafixer.carry_forward_forcs()

Copying 2015 FOR codes to 2018 for unhandled rows
Found 250 total unhandled rows
Moving 4 values to for2_e18
Moving 0 values to for3perc_e18
Moving 0 values to for3_e18
Moving 54 values to for1_e18
Moving 1 values to for4_e18
Moving 4 values to for2perc_e18
Moving 1 values to for4perc_e18
Moving 54 values to for1perc_e18


In [14]:
astro_erafixer.save()

File saved: disc_astro.xlsx


'disc_astro.xlsx'

#### Apply FORC_STRING

In [15]:
astro_erafixer = EraFixer(fn='disc_astro.xlsx', verbose=True)

Parsing file disc_astro.xlsx
Using sheet index 0 - Sheet1


In [16]:
forc_string = '0201'

astro_erafixer.set_forc_string(forc_string)

Applying FORC_STRING '0201'


In [17]:
astro_erafixer.save()

File saved: disc_astro.xlsx


'disc_astro.xlsx'

##### More examples

In [18]:
# Other examples
photonics_erafixer = EraFixer(fn='disc_photonics.xlsx', verbose=True)

author = 'spence'

forc_string = '0205'
# forc_string = '0201:40,0203'
# forc_string = '0201:40,0203:60'
# forc_string = '0201:40,0203:30,0204'
# forc_string = '0201:40,0203:25,0204:30'

# justify_string = None
justify_string = 'Development of Raman lasers'

photonics_erafixer.set_forc_string(forc_string, justify_string=justify_string, author=author)
photonics_erafixer.save()

Parsing file disc_photonics.xlsx
Using sheet index 0 - Sheet1
Applying FORC_STRING '0205'
File saved: disc_photonics.xlsx


'disc_photonics.xlsx'

### Test author match

We want to make sure we are matching the correct authors. The author should be supplied as the full last name of the author in question. Names are not stored correctly in the excel sheet (should be in UTF-8) but will still match on bad characters. See top name below.

In [19]:
authors = [
    'JelÃ­nkovÃ¡',
    'gee',
    'McGee',
    '    sTeEl ',
    'steel',
    'steele',
    'casteels',
    'xia',
    'xiao',
    'Zvyagin'
]

for author in authors:
    
    row_match = erafixer.get_matching_rows(author, 'AUTHORS', blank_discipline=False)
    print(row_match)
    if(row_match):
        display_helpers.show_matches(erafixer, row_match, author)

[242, 268, 1365, 1492]


Full Name Jelã­Nkovã¡ Helena     Authors: JelÃ­nkovÃ¡ Helena;Kitzler OndÂ¿ej;JelÃ­nek Michal;Â¿ulc Jan;NÂ¿mec Michal;KubeÂ¿ek VÃ¡clav Title: Diamond Raman laser in eye safe region ---   Authors: Kitzler OndÂ¿ej;JelÃ­nkovÃ¡ Helena;Â¿ulc Jan;KoubÃ­kovÃ¡ Lucia;NÂ¿mec Michal;Nejezchleb Karel;Â¿koda VÃ¡clav Title: High energy intracavity pumped eye-safe BaWOÂ¿ Raman laser ---   Authors: JelÃ­nek Michal;Kitzler OndÂ¿ej;JelÃ­nkovÃ¡ Helena;Â¿ulc Jan;NÂ¿mec Michal;KubeÂ¿ek VÃ¡clav Title: CVD-diamond external cavity Raman laser operating at 1632 nm ---   Authors: JelÃ­nek M.;Kitzler O.;JelÃ­nkovÃ¡ H.;Â¿ulc J.;NÂ¿mec M. Title: CVD-diamond external cavity nanosecond Raman laser operating at 1.63 Â¿m pumped by 1.34 Â¿m Nd:YAP laser ---

[1374]


Full Name Gee Wilfred T.     Authors: Gee Wilfred T.;Guyon Olivier;Walawender Josh;Jovanovic Nemanja;Boucher Luc Title: Project PANOPTES a citizen-scientist exoplanet transit survey using commercial digital cameras ---

[29, 374]


...

Full Name Xia Ke Yu     Authors: Xia Ke Yu ; Zhao Nan ; Twamley Jason Title: Detection of a weak magnetic field via cavity-enhanced Faraday rotation ---   Authors: Xia Ke Yu ; Twamley Jason Title: Solid-state optical interconnect between distant superconducting quantum chips ---   Authors: Xia Ke Yu ; Lu Guowei ; Lin Gongwei ; Cheng Yuqing ; Niu Yueping ; Gong Shangqing ; Twamley Jason Title: Reversible nonmagnetic single-photon isolation using unbalanced quantum coupling ---   Authors: Xia Keyu;Brennen Gavin K.;Ellinas Demosthenes;Twamley Jason Title: Deterministic generation of an on-demand photon fock state from a solid-state system ---

[273, 872, 1366]


Full Name Che Xiao     Authors: Che Xiao;Sturmann Laszlo;Monnier John D.;ten Brummelaar Theo A.;Sturmann Judit;Ridgway Stephen T.;Ireland Michael J.;Turner Nils H.;McAlister Harold A. Title: The CHARA array adaptive optics I common-path optical and mechanical design, and preliminary on-sky results ---   Authors: Bundy Kevin ; Bershady Matthew A. ; Law David ; Yan Renbin ; Drory Niv ; MacDonald Nicholas R. ; Wake David A. ; Cherinka Brian A. ; Sanchez-Gallego Jose Ramon ; Weijmans Anne-Marie ; Thomas Daniel B. ; Tremonti Christy A ; Masters Karen Louise ; Coccato Lodovico ; Diamond-Stanic Aleksandar M. ; Aragon-Salamanca Alfonso ; Avila-Reese Vladimir ; Badenes Carlos ; Falcon-Barroso Jesus ; Belfiore Francesco ; Bizyaev Dmitriy V ; Blanc Guillermo A. ; Bland-Hawthorn Jonathan ; Blanton Michael R ; Brownstein Joel R. ; Byler Nell ; Cappellari Michele ; Conroy Charlie ; Dutton Aaron A. ; Emsellem Eric ; Etherington James ; Frinchaboy Peter M. ; Fu Hai ; Gunn James E ; Harding Paul ; Johnston Evelyn J. ; Kauffmann Guinevere ; Kinemuchi Karen ; Klaene Mark A. ; Klaene Mark A. ; Leauthaud Alexie ; Li Cheng ; Lin Lihwai ; Maiolino Roberto ; Malanushenko Viktor ; Malanushenko Elena ; Mao Shude ; Maraston C ; McDermid Richard ; Merrifield Michael R. ; Nichol Robert C. ; Oravetz Daniel J. ; Pan Kaike ; Parejko John K. ; Sanchez Sebastian F. ; Schlegel David J ; Simmons Audrey E. ; Steele Oliver ; Steinmetz Matthias ; Thanjavur Karun ; Thompson Benjamin A. ; Tinker Jeremy L. ; van den Bosch Remco C. E. ; Westfall Kyle B. ; Wilkinson David ; Wright Shelley ; Xiao Ting ; Zhang Kai Title: Overview of the SDSS-IV MaNGA survey : mapping nearby galaxies at Apache Point Observatory ---   Authors: Ten Brummelaar Theo A.;Sturmann Laszlo;Sturmann Judit;Ridgway Stephen T.;Monnier John D.;Ireland Michael J.;Che Xiao;McAlister Harold A.;Turner Nils H.;Tuthill P. G. Title: Adaptive optics for the CHARA array ---

[50, 57, 70, 156, 203, 738, 740, 747, 1088, 1115, 1117, 1121, 1123, 1126, 1132, 1147, 1155, 1170, 1171, 1199, 1202, 1207, 1229, 1239, 1274, 1278, 1282, 1295, 1298, 1319, 1323, 1328, 1481, 1487, 1493, 1505, 1579, 1597, 1632]


Full Name Zvyagin Andrei V.     Authors: Razali Wan A. W.;Sreenivasan Varun K.A.;Bradac Carlo;Connor Mark;Goldys Ewa M.;Zvyagin Andrei V. Title: Wide-field time-gated photoluminescence microscopy for fast ultrahigh-sensitivity imaging of photoluminescent probes ---   Authors: Guller Anna;Nadort Annemarie;Generalova Alla;Kornienko Inna;Petersen Elena;Qian Yi;Shekhter Anatoly;Goldys Ewa;Zvyagin Andrei Title: Onion-like surface design of upconverting nanophosphors modified with polyethylenimine shielding toxicity versus keeping brightness? ---   Authors: Zvyagin A. V.;Sreenivasan V. K A;Kelf T. A.;Deyev S. M. Title: Luminescent nanodiamonds for intracellular imaging ---   Authors: Popov Alexey P.;Bykov Alexander V.;Sokolov Victor I.;Lysak Yulia V.;Nadort Annemarie;Priezzhev Alexander V.;MyllylÃ¤ Risto;Zvyagin Andrei V. Title: Upconversion luminophores as a novel tool for deep tissue imaging ---